Identifying stonewalls

In [39]:
# Local path, change this.
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
import pandas as pd
import ml_utils
import math
import time
import numpy as np
import os

np.set_printoptions(suppress=True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Dropout, Conv2D, MaxPooling2D, Flatten, Conv2DTranspose, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.utils import plot_model

folder = "C:/Users/caspe/Desktop/wall_data/"

In [3]:
dtm = np.load(folder + "dtm.npy")
walls = np.load(folder + "walls.npy")
meta = pd.read_csv(folder + "metadata.csv")

In [12]:
walls = (walls == 1).astype('uint8')

In [5]:
land_mask = np.swapaxes(meta[meta["land"] == 1].values, 0, 1)[0]
wall_mask = np.swapaxes(meta[meta["wall"] == 1].values, 0, 1)[0]    # Has a wall
no_wall_mask = np.swapaxes(meta[meta["wall"] == 2].values, 0, 1)[0] # Has _no_ wall

In [8]:
no_wall_mask.shape

(715,)

In [7]:
wall_mask.shape

(599,)

Regression first

In [25]:
train = np.concatenate([dtm[wall_mask], dtm[no_wall_mask]])

In [24]:
truth = np.concatenate([meta.values[wall_mask], meta.values[no_wall_mask]])

In [26]:
truth = (np.swapaxes(truth, 0, 1)[2] == 1).astype('uint8')

(1314, 3)

In [36]:
train = ml_utils.add_rotations(train)
train.shape

(5256, 64, 64)

In [37]:
truth = np.concatenate([truth, truth, truth, truth])

In [38]:
# Shuffle the dataset
shuffle_mask = np.random.permutation(len(truth))
train = train[shuffle_mask]
truth = truth[shuffle_mask]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train, truth, test_size=0.33, random_state=42)

In [ ]:
def define_model(shape, name, activation='relu', kernel_initializer='normal', maxnorm=4, sizes=[64, 96, 128]):
    model_input = Input(shape=shape, name=name)
    model = Conv2D(sizes[0],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
        kernel_constraint=max_norm(maxnorm),
        bias_constraint=max_norm(maxnorm),
    )(model_input)
    model = BatchNormalization()(model)

    model = Conv2D(sizes[1],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
        kernel_constraint=max_norm(maxnorm),
        bias_constraint=max_norm(maxnorm),
    )(model)
    model = BatchNormalization()(model)

    model = Conv2D(sizes[2],
        kernel_size=3,
        padding='same',
        activation=activation,
        kernel_initializer=kernel_initializer,
        kernel_constraint=max_norm(maxnorm),
        bias_constraint=max_norm(maxnorm),
    )(model)
    model = BatchNormalization()(model)

    # Switch this to regression
    output = Conv2D(1, kernel_size=3, padding='same', activation='relu', kernel_initializer=kernel_initializer)(model)

    return Model(inputs=[model_input], outputs=output)

lr = 0.001
bs = 512
epochs = 50

def step_decay(epoch):
    initial_lrate = lr
    drop = 0.5
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

model = define_model(X_train.shape[1:], "Generative")

model.compile(
    optimizer=Adam(
        learning_rate=lr,
        name="Adam",
    ),
    loss='log_cosh',
    metrics=[
        'mse',
        'mae',
        'log_cosh',
    ]
)

model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    verbose=0,
    batch_size=bs,
    validation_split=0.2,
    callbacks=[
        LearningRateScheduler(step_decay),
        EarlyStopping(
            monitor="val_loss",
            patience=10,
            min_delta=0.1,
            restore_best_weights=True,
        ),
    ],
)

print(f"Batch_size: {str(bs)}")
loss, mse, mae, log_cosh = model.evaluate(X_test, y_test, verbose=0)
print(f"Mean Square Error:      {round(mse, 3)}")
print(f"Mean Absolute Error:    {round(mae, 3)}")
print(f"log_cosh:               {round(log_cosh, 3)}")
print("")

import pdb; pdb.set_trace()